In [1]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
#from matplotlib_venn import venn2, venn2_circles
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import scipy
import lightgbm as lgb
from scipy.sparse import hstack, csr_matrix

In [2]:

train = pd.read_csv('train.csv')
train_active = pd.read_csv('train_active.csv')
test = pd.read_csv('test.csv')
test_active = pd.read_csv('test_active.csv')

train_periods = pd.read_csv('periods_train.csv', parse_dates=['date_from', 'date_to'])
test_periods = pd.read_csv('periods_test.csv', parse_dates=['date_from', 'date_to'])

train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [14]:
test_active = pd.read_csv('test_active.csv')


In [21]:
train_periods['total_period'] = train_periods['date_to'] - train_periods['date_from']

In [5]:
datelist = pd.date_range(train_periods['date_from'].min(), train_periods['date_to'].max()).tolist()

In [3]:
train_periods['date_to'] = pd.to_datetime(train_periods['date_to'])
train_periods['date_from'] = pd.to_datetime(train_periods['date_from'])

In [ ]:
train_periods['date_to'] = pd.to_datetime(train_periods['date_to'])
train_periods['date_from'] = pd.to_datetime(train_periods['date_from'])

In [ ]:
train_periods['activation_date'] = pd.to_datetime(train_periods['activation_date'])

In [30]:
train_periods.head()

,item_id,activation_date,date_from,date_to,total_period
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16,1 days
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18,3 days
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28,13 days
3,80bf58082ad3,2017-03-19,2017-03-19,2017-03-28,9 days
4,67a9944a7373,2017-03-14,2017-03-15,2017-03-28,13 days


In [ ]:
from tqdm import tqdm_notebook as tqdm
def check_date(dfrom, dto, dt):
    checker = (dfrom <= dt) and (dto >= dt)
    return checker
for dt in tqdm(datelist[:1]):
    #train_periods["day_"+str(dt.day)] = train_periods.apply(lambda x: (x['date_from'] < dt) and (x['date_to'] > dt),
    #                                                        axis=1)
    train_periods["day_"+str(dt.day)] = train_periods.apply(lambda x: check_date(x['date_from'], x['date_to'], dt),
                                                           axis=1)

In [21]:
all_periods = pd.concat([
    train_periods,
    test_periods
])

del train_periods
del test_periods
gc.collect()

all_periods.head()

/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


,item_id,activation_date,date_from,date_to
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28
3,80bf58082ad3,2017-03-19,2017-03-19,2017-03-28
4,67a9944a7373,2017-03-14,2017-03-15,2017-03-28


In [22]:
len(all_periods)

30412334

In [26]:
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
tqdm.pandas()
num_partitions = 15
num_cores = multiprocessing.cpu_count()

def parallelize_dataframe(df, func, dt):
    a,b,c,d,e,f,g,h,i,j,k,l,m,n,o = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.starmap(func, [(a, dt),(b,dt),(c, dt),(d, dt),(e, dt),(f, dt),(g, dt),(h,dt),(i,dt),(j,dt),
                                      (k,dt),(l,dt),(m,dt),(n,dt),(o,dt) ]))
    pool.close()
    pool.join()
    return df

In [27]:
def check_date(dfrom, dto, dat):
    checker = (dfrom <= dat) and (dto >= dat)
    return int(checker)
def get_date_info(df, dt):
    date_data = []
    for idx, c_row in tqdm(df.iterrows(), total=len(df)):
        date_data.append(check_date(c_row['date_from'], c_row['date_to'], dt))
    df["day_"+str(dt.day)] = date_data
    return df

In [28]:
for dt in datelist:
    all_periods = parallelize_dataframe(all_periods, get_date_info, dt)

  0%|          | 0/2027489 [00:00<?, ?it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 784/2027489 [00:11<8:13:48, 68.40it/s] /home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 1291/2027489 [00:11<5:13:22, 107.76it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 4925/2027489 [00:15<1:47:52, 312.49it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tq

  6%|▋         | 128547/2027489 [00:29<07:12, 4392.70it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  7%|▋         | 142139/2027489 [00:31<06:51, 4579.38it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 0/2027489 [00:00<?, ?it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 6099/2027489 [00:11<1:01:48, 545.01it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm

  0%|          | 8880/2027489 [00:11<43:52, 766.92it/s]  /home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  1%|          | 12594/2027489 [00:12<33:32, 1001.38it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  1%|▏         | 29628/2027489 [00:15<17:11, 1937.70it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  1%|▏         | 30303/2027489 [00:15<17:03, 1951.67it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://git

  5%|▍         | 94404/2027489 [00:27<09:31, 3381.25it/s]]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 0/2027489 [00:00<?, ?it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  1%|          | 12277/2027489 [00:12<32:50, 1022.90it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  1%|          | 13632/2027489 [00:12<30:02, 1117.16it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/

  1%|          | 22222/2027489 [00:13<20:13, 1653.12it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  1%|          | 12262/2027489 [00:12<34:58, 960.15it/s]]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 6951/2027489 [00:12<1:02:09, 541.84it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  2%|▏         | 45923/2027489 [00:16<12:05, 2731.04it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://git

  0%|          | 0/2027489 [00:00<?, ?it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 9223/2027489 [00:11<42:50, 785.28it/s]  /home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 1211/2027489 [00:11<5:07:37, 109.78it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  1%|          | 13278/2027489 [00:12<31:27, 1067.18it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/t

In [29]:
all_periods.head()

,item_id,activation_date,date_from,date_to,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18,1,1,1,1,0,0,0,0,0,0,0,0,0,0
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,80bf58082ad3,2017-03-19,2017-03-19,2017-03-28,0,0,0,0,1,1,1,1,1,1,1,1,1,1
4,67a9944a7373,2017-03-14,2017-03-15,2017-03-28,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [28]:
train_periods['date_from'].max()-train_periods['date_from'].min()

Timedelta('13 days 00:00:00')

In [92]:
#used_cols = ['item_id', 'user_id', 'category_name', 'parent_category_name', 'city','region', 'user_type',
#             'param_1','param_2']
used_cols = ['item_id', 'param_1', 'param_2']
train = pd.read_csv('train.csv', usecols=used_cols)
train_active = pd.read_csv('train_active.csv', usecols=used_cols)
test = pd.read_csv('test.csv', usecols=used_cols)
test_active = pd.read_csv('test_active.csv', usecols=used_cols)

/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [93]:
all_samples = pd.concat([
    train,
    train_active,
    test,
    test_active
]).reset_index(drop=True)
all_samples.drop_duplicates(['item_id'], inplace=True)

del train_active
del test_active
gc.collect()

/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


2309

In [94]:
all_periods = all_periods.merge(all_samples, on='item_id', how='left')


/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [95]:
all_periods.head()

,item_id,activation_date,date_from,date_to,day_15,day_16,day_17,day_18,day_19,day_20,...,day_28,user_id,region,city,parent_category_name,category_name,user_type,days_to_display,param_1,param_2
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16,1,1,0,0,0,0,...,0,e292cce69842,Нижегородская область,Нижний Новгород,Услуги,Предложение услуг,Private,29.0,"Транспорт, перевозки",Спецтехника
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18,1,1,1,1,0,0,...,0,a326c04a24ec,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Company,27.0,Для мальчиков,Другое
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28,1,1,1,1,1,1,...,1,06d275498a56,Свердловская область,Екатеринбург,Личные вещи,"Одежда, обувь, аксессуары",Company,14.0,Женская одежда,Обувь
3,80bf58082ad3,2017-03-19,2017-03-19,2017-03-28,0,0,0,0,1,1,...,1,831c8c4a622c,Краснодарский край,Краснодар,Для дома и дачи,Бытовая техника,Private,0.0,Для кухни,Холодильники и морозильные камеры
4,67a9944a7373,2017-03-14,2017-03-15,2017-03-28,1,1,1,1,1,1,...,1,248102e50d79,Пермский край,Пермь,Личные вещи,Товары для детей и игрушки,Shop,1.0,Игрушки,NaN


/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [96]:
all_periods.to_csv('all_periods.csv', index=False)

/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [62]:
all_periods['activation_date'] = pd.to_datetime(all_periods['activation_date'])

In [63]:
all_periods['days_to_display'] = (all_periods['date_from'] - all_periods['activation_date']).dt.days

In [45]:
sel_cols = []
sel_cols.append('category_name')
for dt in datelist:
    sel_cols.append('day_'+str(dt.day))

In [65]:
random = all_periods[['category_name','days_to_display']].groupby(['category_name'])[['days_to_display']].mean().reset_index()

In [75]:
gp = all_periods[sel_cols].groupby([sel_cols[0]])[sel_cols[1:]].sum().reset_index()
gp.head()

,category_name,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28
0,Автомобили,313710,319370,314148,308983,303424,299651,296367,293020,288685,285579,282870,280588,278917,269624
1,Аквариум,39952,40445,40315,39933,39688,39264,39154,38981,38741,38537,38476,38342,37946,37427
2,Аудио и видео,228110,230071,228575,226354,224661,223628,222411,221330,219617,218055,216841,216282,215030,211361
3,Билеты и путешествия,11017,11187,10983,10583,10320,10180,10040,9963,9743,9534,9284,9144,9055,8803
4,Бытовая техника,192985,195019,192572,190309,188602,187459,186370,184882,182891,181257,179952,179377,178504,174856


In [76]:
gp = gp.merge(random, on='category_name', how='left')


In [82]:
gp.head()

,category_name,days_to_display,mean_weekday,max_weekday,min_weekday,std_weekday,mean_weekend,max_weekend,min_weekend,std_weekend,mean,max,min,std
0,Автомобили,9.539334,295907.1,319370,269624,16196.838584,293966.25,308983,280588,14341.735747,295352.571429,319370,269624,15162.851724
1,Аквариум,10.573292,39076.2,40445,37427,978.999353,39109.75,39933,38342,817.148038,39085.785714,40445,37427,904.364050
2,Аудио и видео,10.575146,221818.8,230071,211361,6070.461923,221034.50,226354,216282,5216.010001,221594.714286,230071,211361,5650.271313
3,Билеты и путешествия,10.496222,10050.5,11187,8803,818.430239,9832.75,10583,9144,724.750704,9988.285714,11187,8803,771.597487
4,Бытовая техника,10.328232,185679.5,195019,174856,6568.889256,184560.00,190309,179377,5700.465420,185359.642857,195019,174856,6135.768231


In [78]:
weekday = []
weekend = []
for dt in datelist:
    if(dt.weekday()<5):
        weekday.append('day_'+str(dt.day))
    else:
        weekend.append('day_'+str(dt.day))
mean_cols = []
for dt in datelist:
    mean_cols.append('day_'+str(dt.day))


In [79]:
gp['mean_weekday'] = gp[weekday].mean(axis=1)
gp['max_weekday'] = gp[weekday].max(axis=1)
gp['min_weekday'] = gp[weekday].min(axis=1)
gp['std_weekday'] = gp[weekday].std(axis=1)
gp['mean_weekend'] = gp[weekend].mean(axis=1)
gp['max_weekend'] = gp[weekend].max(axis=1)
gp['min_weekend'] = gp[weekend].min(axis=1)
gp['std_weekend'] = gp[weekend].std(axis=1)


In [80]:
gp['mean'] = gp[mean_cols].mean(axis=1)
gp['max'] = gp[mean_cols].max(axis=1)
gp['min'] = gp[mean_cols].min(axis=1)
gp['std'] = gp[mean_cols].std(axis=1)

In [81]:
for col in mean_cols:
    gp.drop(col,inplace=True, axis=1)

In [83]:
gp.to_csv('columns_per_day_stats.csv')

In [97]:
#########################################AUTOMATION######################################
get_feat_col = ['category_name','parent_category_name', 'city','region', 'user_type','param_1','param_2']
for get_fc in tqdm(get_feat_col):
    sel_cols = []
    sel_cols.append(get_fc)
    for dt in datelist:
        sel_cols.append('day_'+str(dt.day))
    gp = all_periods[sel_cols].groupby([sel_cols[0]])[sel_cols[1:]].sum().reset_index()
    random = all_periods[[get_fc,'days_to_display']].groupby([get_fc])[['days_to_display']].mean().reset_index()
    gp = gp.merge(random, on=get_fc, how='left')
    gp['mean_weekday'] = gp[weekday].mean(axis=1)
    gp['max_weekday'] = gp[weekday].max(axis=1)
    gp['min_weekday'] = gp[weekday].min(axis=1)
    gp['std_weekday'] = gp[weekday].std(axis=1)
    gp['mean_weekend'] = gp[weekend].mean(axis=1)
    gp['max_weekend'] = gp[weekend].max(axis=1)
    gp['min_weekend'] = gp[weekend].min(axis=1)
    gp['std_weekend'] = gp[weekend].std(axis=1)
    gp['mean'] = gp[mean_cols].mean(axis=1)
    gp['max'] = gp[mean_cols].max(axis=1)
    gp['min'] = gp[mean_cols].min(axis=1)
    gp['std'] = gp[mean_cols].std(axis=1)
    for col in mean_cols:
        gp.drop(col,inplace=True, axis=1)
    gp.to_csv(get_fc+'_per_day_stats.csv', index=False)




  0%|          | 0/7 [00:00<?, ?it/s]/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)

100%|██████████| 7/7 [01:49<00:00, 15.62s/it]


In [98]:
gp.head()

,param_2,days_to_display,mean_weekday,max_weekday,min_weekday,std_weekday,mean_weekend,max_weekend,min_weekend,std_weekend,mean,max,min,std
0,1,11.304857,152282.7,161626,143234,6382.759087,149363.00,155943,144332,5755.212768,151448.500000,161626,143234,6141.786770
1,2,11.312225,154936.9,163246,146742,5683.191151,152236.75,158072,147778,5037.846456,154165.428571,163246,146742,5460.758833
2,2112,NaN,1.0,1,1,0.000000,1.00,1,1,0.000000,1.000000,1,1,0.000000
3,3,11.374827,101123.4,105796,96620,3159.028022,99504.75,102736,97043,2780.356257,100660.928571,105796,96620,3044.440822
4,4,11.371056,18134.5,18871,17525,476.509590,17879.50,18418,17533,424.770134,18061.642857,18871,17525,461.654817


/home/user/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [ ]:
for dt in datelist[1:]:
    df["day_"+str(dt.day)] = 

In [ ]:
n_user_items = all_samples.groupby(['user_id'])[['item_id']].count().reset_index() \
    .rename(index=str, columns={
        'item_id': 'n_user_items'
    })
gp = gp.merge(n_user_items, on='user_id', how='left')


In [16]:
test_active.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type
0,a66b215700dc,ef734a1653a8,Воронежская область,Борисоглебск,Личные вещи,Детская одежда и обувь,Для девочек,Брюки,86-92 см (1-2 года),Джинсы,Джинсы на девочку. В отличном состоянии.,300.0,58.0,2017-04-12,Private
1,4d0e7002bda8,b15c020d9f61,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,Усилители и ресиверы,NaN,NaN,Luxman C-5000 A с мс трансформатором,Топовый предусилитель с очень качественным пор...,100000.0,18.0,2017-04-13,Company
2,da2ca7a9f2db,8b8c67de263b,Белгородская область,Старый Оскол,Хобби и отдых,Коллекционирование,Другое,NaN,NaN,"Шахматы ""Властелин колец"" первые 13 выпусков","Продам шахматы ""Властелин колец"" первые 13 вып...",NaN,1.0,2017-04-12,Private
3,6367be7c6025,07724ae38fc3,Свердловская область,Екатеринбург,Хобби и отдых,Книги и журналы,Книги,NaN,NaN,Вампирские тайны Шарлин Харрис романы,"11 книг, серия Вампирские тайны, Шарлин Харрис...",130.0,27.0,2017-04-12,Private
4,446f4d79a77a,db71e33ba4b6,Кемеровская область,Кемерово,Личные вещи,Детская одежда и обувь,Для мальчиков,Трикотаж,74-80 см (7-12 мес),Продам комбинезон,"Комбинезон крокид на рост 74 см,мы носили с 68...",400.0,4.0,2017-04-12,Private


- Count of number of same category by regions that are active
- Count of number of same category by city that are active
- Count of number of same parent_category_name by regions that are active
- Count of number of same parent_category_name by city that are active
- Count of number of same param1 by regions that are active
- Count of number of same param1 by city that are active
- Count of number of same param2 by regions that are active
- Count of number of same param2 by city that are active

- Predict classes of resnet and probability
- Predict classes of inception_resnet and probability
- Count of number of same category by category resnet that are active
- Count of number of same category by category inception that are active
- Ensemble resnet and inception get category and probability
- Count of number of same category by ensembled category that are active
- get bin groups of price and categorize the data
- count of number of same price category that are active
- 